# A simple stochastic process for ion scattering

#### Idea

Suppose an ion has to traverse a medium (homogeneous or inhomogeneous), starting at some point with some initial velocity and moving in 2 dimensions, the $(z, y)$ plane, where $z$ is the transverse direction w.r.t. the medium. The particle "goes forward" along the $z$ axis and "goes up or down" along the $y$ axis. The medium extends in $[z_I, z_F]$ in the $z$ direction and indefinitely in the $y$ direction.

We assume we can measure the $y$ coordinate of the particles in $z_I$ and $z_F$, with $y_I = y(z_I)$ and $y_F = y(z_F)$ and the corresponding velocity vectors $\mathbf{v}_I = \mathbf{v}(z_I)$ and $\mathbf{v}_F = \mathbf{v}(z_F)$, with $\mathbf{v} \equiv \left(\begin{array}{c} v_z \\ v_y \end{array}\right)$. We also assume the medium is homogeneous in the $y$ direction so that the inhomogeneity (if any) only happens as the ion moves forward.

#### Coordinate system

We parametrize the trajectory $\mathbf{s} \equiv \left(\begin{array}{c} z \\ y \end{array}\right)$ as a function of $z$ itself (rather than time), which we discretize into an evenly-spaced grid $z_0 \equiv z_I, z_1, z_2, \ldots, z_{N-1}, z_N \equiv z_F$ so that the problem boils down to finding the function $y(z)$, i.e. the sequence $y_1 = y(z_1), \ldots, y_{N-1} = y(z_{N-1})$ (we already know $y_0 \equiv y_I$ and $y_N \equiv y_F$).

We define $\theta\in[-\pi, \pi]$ to be the angle between $\mathbf{v}$ and the $z$ axis (the range of $\theta$ has been chosen to conform to the convention often used for angular probability distributions).

#### Dynamics

We usa a simple model for the dynamics of the ion traversing the medium:
- At each $z$, the ion interacts with the medium with some probability, which we assume to be Bernoulli with some probability parameter $p$:
$$
p(\mathrm{interaction} | z) = \mathrm{Ber}( x = 1 | p(z)),
$$
where $x$ is a label indicating if the interaction takes place ($x=1$) or not ($x=0$). We assume the parameter $p$ is in a function of $z$, with the idea that more "dense" regions of the medium give a higher probaibility of interaction (bigger $p$) while less dense regions give a lower probability (smaller $p$). If we assume the medium is homogeneous, then $p$ is a constant.

- If an interaction takes place, we assume the angle $\theta$ varies as $\theta \to \theta + \delta\theta$, where
$$
p(\delta\theta | z) = f(\delta\theta | \mu = 0, \kappa(z)) \equiv \frac{\exp\left( \kappa \cos(\delta\theta - \mu) \right)}{2\pi I_0(\kappa)}
$$
is the von Mises distribution with parameters $\mu\in\mathbb{R}$ and $\kappa \in (0, +\infty)$ (a sort of angular version of a Gaussian), $I_0(\kappa)$ being the modified Bessel function of the first kind of order $\kappa$. The distribution is symmetric around its $\mu$, which also sets its maximum, and $\kappa$ parametrizes its variance, and is conveniently implemented in as the `VonMises` distribution in Tensorflow Probability, with parameters `loc`$\equiv \mu$ and `concentration`$\equiv\kappa$. In the general case of inhomogeneous media, we assume $\kappa = \kappa(z)$ to reflect that some regions can deflect the with bigger angles than others. We assume no preferred direction for the deviation, so we keep $\mu = 0$. In the case of an homogeneous medium, $\kappa$ becomes a constant as well.

- The ion travels in a straight line depending on its velocity if no interaction takes place.

**Notes:**
- If we parametrize $y$ as a function of $z$, work with a grid (1-dimensional lattice) of $z$ values **and** impose that at every evolution step $z$ increases of one lattice space $\Delta$, irrespective of $v_z$, we are in fact using a discretized time with **non-constant spacing**. Meaning: we update the system (and check if an interaction happens) every time $z$ increases by $\Delta$, but the physical time the system takes to undergo such evolution vaies.
- The above time depends on $v_z$ as $\Delta t = \Delta / v_z$.
- This way of doing things is a simplification and **doesn't** cater for the cases in which the ion is scattered backwards by the medium ($|\delta\theta| > \pi/2$).
- A better way of doing things would be to have discrete time and impose some probability of interaction at each time step (maybe as a Poisson point process, in which the probability of a given time inteval between two subsequent interactions follows an exponential distribution).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_probability as tfp

tfd = tfp.distributions

sns.set_theme()

In [ ]:
def update_position(s, v, delta=1.):
    """
    Updates positions assuming that the displacement along
    the z direction (`delta`) is constant.
    """
    z, y = s
    v_z, v_y = v

    delta_z = delta
    delta_y = delta_z * v_y / v_z

    return (z + delta_z, y + delta_y)


def update_velocity(v, delta_theta):
    """
    Update the velocity given the variation `delta_theta` in
    its angle w.r.t. the z axis. The norm of the velocity is
    assumed to remain constant (elastic scattering).
    """
    v_z, v_y = v
    
    v_norm = np.linalg.norm(v_i)
    theta = np.arctan(v_y / v_z)

    v_y_new = v_norm * np.sin(theta + delta_theta)
    v_z_new = v_norm * np.cos(theta + delta_theta)

    return (v_z_new, v_y_new)


def generate_evolution(s_i, v_i, n_steps, p_interaction, kappa, delta=1.):
    """
    """
    s = [s_i]
    v = [v_i]

    delta_theta_samples = tfd.Mixture(
        cat=tfd.Categorical(probs=[p_interacion, 1.-p_interacion]),
        components=[
            tfd.VonMises(loc=0., concentration=kappa),
            tfd.Deterministic(loc=0.),
        ]
    ).sample(n_steps)
    
    
    for i in range(n_steps):
        s.append(update_position(s[-1], v[-1], delta=delta))
    
        v.append(update_velocity(v[-1], delta_theta_samples[i]))
    
    return np.array(s)

In [ ]:
n_histories = 20

s = np.array([
    generate_evolution(
        s_i=(0., 0.),
        v_i=(0.1, 0.1),
        n_steps=100,
        p_interaction=0.2,
        kappa=20,
        delta=1.
    )
    for _ in range(n_histories)
])


fig = plt.figure(figsize=(14, 6))

for i in range(n_histories):
    sns.scatterplot(
        x=s[i, :, 0],
        y=s[i, :, 1]
    )